# 2024-10-21 YCB Visualization

The purpose of this notebook is to make **organized** visualizations of the YCB dataset (77 objects, 32 rotations).

It would be nice to create the following visualizations for reference:
- For each object, show all 32 rotations in a 4 x 8 grid.
- For all objects, show 4 x 77 grid of 4 selected rotations each. 

In [1]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import tqdm

data_dir = "../data/view_finder_224/eval/view_finder_rgbd"
visualization_dir = "../data/view_finder_224/eval/view_finder_rgbd/visualizations"
os.makedirs(visualization_dir, exist_ok=True)

In [2]:
# Load episodes.json to get object ids and rotations
episodes = []
with open(os.path.join(data_dir, "episodes.json"), "r") as f:
    for line in f:
        episode = json.loads(line)
        episode_num = episode["episode"]
        object = episode["object"]
        rotation = episode["rotation"]
        episodes.append((episode_num, object, rotation))

print(episodes[:3])


[(0, 'mug', [0, 0, 0]), (1, 'bowl', [0, 0, 0]), (2, 'potted_meat_can', [0, 0, 0])]


In [3]:
# Let's see unique rotations
rotations = [tuple(episode[2]) for episode in episodes]
unique_rotations = set(rotations)
print(unique_rotations)
print(len(unique_rotations))


{(0, 0, 270), (0, 0, 90), (0, 90, 90), (0, 90, 270), (90, 180, 90), (90, 180, 270), (90, 270, 90), (90, 270, 270), (0, 180, 90), (0, 270, 90), (0, 180, 270), (0, 270, 270), (90, 90, 270), (90, 0, 0), (90, 0, 180), (90, 90, 0), (90, 90, 180), (0, 0, 180), (0, 90, 0), (0, 0, 0), (0, 90, 180), (90, 180, 0), (90, 180, 180), (90, 90, 90), (90, 270, 0), (90, 270, 180), (0, 180, 0), (0, 180, 180), (0, 270, 0), (0, 270, 180), (90, 0, 90), (90, 0, 270)}
32


In [4]:
# Let's get unique objects so we can create 4 x 8 grids for each object.
objects = [episode[1] for episode in episodes]
unique_objects = set(objects)
print(unique_objects)
print(len(unique_objects))


{'f_lego_duplo', 'nine_hole_peg_test', 'b_lego_duplo', 'mug', 'd_toy_airplane', 'mustard_bottle', 'b_cups', 'e_toy_airplane', 'pear', 'hammer', 'h_cups', 'a_toy_airplane', 'pudding_box', 'knife', 'apple', 'wood_block', 'c_cups', 'gelatin_box', 'e_cups', 'd_cups', 'b_marbles', 'foam_brick', 'a_colored_wood_blocks', 'plate', 'g_cups', 'padlock', 'orange', 'skillet_lid', 'racquetball', 'sponge', 'banana', 'phillips_screwdriver', 'f_cups', 'sugar_box', 'lemon', 'mini_soccer_ball', 'plum', 'd_lego_duplo', 'b_colored_wood_blocks', 'tennis_ball', 'medium_clamp', 'c_toy_airplane', 'tuna_fish_can', 'scissors', 'a_cups', 'cracker_box', 'bowl', 'flat_screwdriver', 'large_marker', 'dice', 'a_marbles', 'b_toy_airplane', 'i_cups', 'a_lego_duplo', 'strawberry', 'peach', 'pitcher_base', 'adjustable_wrench', 'chain', 'potted_meat_can', 'c_lego_duplo', 'rubiks_cube', 'e_lego_duplo', 'extra_large_clamp', 'fork', 'baseball', 'spatula', 'tomato_soup_can', 'g_lego_duplo', 'j_cups', 'softball', 'master_chef_

In [5]:
sorted(unique_rotations)

[(0, 0, 0),
 (0, 0, 90),
 (0, 0, 180),
 (0, 0, 270),
 (0, 90, 0),
 (0, 90, 90),
 (0, 90, 180),
 (0, 90, 270),
 (0, 180, 0),
 (0, 180, 90),
 (0, 180, 180),
 (0, 180, 270),
 (0, 270, 0),
 (0, 270, 90),
 (0, 270, 180),
 (0, 270, 270),
 (90, 0, 0),
 (90, 0, 90),
 (90, 0, 180),
 (90, 0, 270),
 (90, 90, 0),
 (90, 90, 90),
 (90, 90, 180),
 (90, 90, 270),
 (90, 180, 0),
 (90, 180, 90),
 (90, 180, 180),
 (90, 180, 270),
 (90, 270, 0),
 (90, 270, 90),
 (90, 270, 180),
 (90, 270, 270)]

In [6]:
def _get_episodes_for_object(object_name, episodes):
    # get all episodes for the given object
    episodes = [episode for episode in episodes if episode[1] == object_name]
    # sort the episodes by rotations
    episodes = sorted(episodes, key=lambda x: x[2])
    return episodes
    

In [7]:
def _plot_all_rotations_for_object(object_name, episodes, visualization_dir):
    # 8 x 4 grid
    fig, axs = plt.subplots(8, 4, figsize=(8, 16))
    for i, episode in enumerate(episodes):
        episode_number = episode[0]
        rotation = episode[2]
        image_rgbd = np.load(os.path.join(data_dir, f"images/{episode_number}.npy"))
        image_rgb = image_rgbd[:, :, :3]
        axs[i // 4, i % 4].imshow(image_rgb)
        # Title with the rotation
        axs[i // 4, i % 4].set_title(f"{rotation}")
        # remove ticks
        axs[i // 4, i % 4].set_xticks([])
        axs[i // 4, i % 4].set_yticks([])

    plt.savefig(os.path.join(visualization_dir, f"{object_name}_8x4.png"), bbox_inches="tight", dpi=300)
    plt.close()


In [8]:
for object in tqdm.tqdm(unique_objects):
    object_episodes = _get_episodes_for_object(object, episodes)
    _plot_all_rotations_for_object(object, object_episodes, visualization_dir)


100%|██████████| 77/77 [01:14<00:00,  1.03it/s]


## 4 x 77 grid